In [ ]:
def splitDataIntoFiles():
    f = open("sf_d9.plain", "r")
    linesPerFile = 100002
    myDir = "sf_d9_data/"

    linesProcessed = 0
    fileNum = 0
    currentFileName = myDir + "sf_d9_0.plain"
    currentFile = open(currentFileName, "w")
    for line in f:
        currentFile.write(line)
        linesProcessed += 1
        if linesProcessed == linesPerFile:
            print("Finished file " + currentFileName)
            linesProcessed = 0
            fileNum += 1
            currentFileName = myDir + "sf_d9_" + str(fileNum) + ".plain"
            currentFile = open(currentFileName, "w")
            
#splitDataIntoFiles()

In [ ]:
import os
fileNames = []
for fileName in os.listdir("sf_d9_data"):
    fileNames.append("sf_d9_data/" + fileName)
numFiles = len(fileNames)
print("Files:", numFiles)

In [ ]:
import random
import numpy as np

def getFensAndScores(fileName):
    f = open(fileName, "r")
    lines = f.readlines()
    leng = len(lines)
    i = 0
    fens = []
    scores = []
    while i < leng and i+2 < leng:
        lines[i] = lines[i].strip() # lines[i] is "fen <fen>"
        lines[i+2] = lines[i+2].strip() # lines[i+2] is "score -529"

        fen = lines[i][4:] # lines[i] is "fen <fen>"

        score = int(lines[i+2][6:]) # lines[i+2] is "score -529"
        if score > 750:
            score = 750
        elif score < -750:
            score = -750
        if " b " in fen:
            score = -score # scores in the file are stm, mirror black ones to get white perspective

        fens.append(fen)
        scores.append(score)
        i += random.randint(10, 20) * 6 # jump to next random fen ("fen <fen>" line is every 6 lines)
    return fens, scores

In [ ]:
fens = []
scores = []
for fileName in fileNames[:250]:
    thisFens, thisScores = getFensAndScores(fileName)
    fens += thisFens
    scores += thisScores

print("Positions:", len(fens))

In [ ]:
import numpy as np
import chess

def fen_to_12x64(fen, debug=False):
    board = chess.Board(fen)
        
    pieces = [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN, chess.KING]
    res = np.zeros((12, 64))
    i = 0 
    for color in [chess.WHITE, chess.BLACK]:
        for piece in pieces:
            bitboard = board.pieces(piece, color).tolist()
            bitboard = np.asarray(bitboard).astype(int)
            if debug:
                print(piece, "white" if color == chess.WHITE else "black")
                print(bitboard.reshape((8,8)))
            res[i] = bitboard
            i += 1
            
    return res

def fen_to_769(fen):
    bbs = fen_to_12x64(fen)
    _768 = bbs.reshape(768, 1)
    _769 = np.append(_768, 1 if " b " in fen else 0)
    return _769

# rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
#fen_to_12x64("7k/7p/6p1/4pp2/4P3/8/3Q1PPP/7K w - - 0 1", debug=True)
;

In [ ]:
from sklearn.model_selection import train_test_split
X_train = [fen_to_769(fen) for fen in fens]
del fens
print("Finished parsing fens to bbs")
X_train, X_test, y_train, y_test = train_test_split(np.array(X_train), np.array(scores), test_size=0.1, random_state=3)
del scores

In [ ]:
import pickle

# mean(abs(tanh(output/400) - tanh(cp/400)))
def custom_loss(y_true, y_pred):
    from tensorflow import reduce_mean, tanh, cast, float32, abs, reduce_sum
    y_true = cast(y_true, dtype=float32)  # Convert y_true to float32
    y_pred = cast(y_pred, dtype=float32)  # Convert y_pred to float32
    tanh_output = tanh(y_pred / 400)
    tanh_cp = tanh(y_true / 400)
    abs_diff = abs(tanh_output - tanh_cp)
    loss = reduce_mean(abs_diff)
    return loss

def train():
    from tensorflow.keras.layers import Dense, Flatten
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping

    model = Sequential()
    #model.add(Flatten(name="flatten", input_shape=(12, 64)))  # Flatten the 3D input to a 1D array
    model.add(Dense(name="dense32", units=32, activation='relu', input_dim=769))
    model.add(Dense(name="output", units=1, activation='linear'))
    model.summary()
    
    model.compile(loss=custom_loss, optimizer=Adam(learning_rate=0.01)) # default lr 0.001
    
    history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2,
                       callbacks=[EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)])
    
    model.save("model.h5")
    with open("history", 'wb') as hist:
        pickle.dump(history.history, hist) 
    
#train()

In [ ]:
from tensorflow.keras.models import load_model
    
model = load_model("model.h5", custom_objects={'custom_loss': custom_loss})
with open("history", "rb") as hist_file:
    history = pickle.load(hist_file)
model.summary()

In [ ]:
def plot(history):
    import matplotlib.pyplot as plt
    if not isinstance(history, dict):
        history = history.history
    # Plot training and validation loss over epochs
    plt.plot(history['loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
plot(history)

In [ ]:
_769 = fen_to_769("rnbk1bnr/pp3Bpp/2p5/4p3/4P3/8/PPP2PPP/RNB1K1NR b KQ - 0 6").reshape(1,769,1)
print(model.predict(_769)[0][0])
_769 = fen_to_769("rnbk1bnr/pp3Bpp/2p5/4p3/4P3/8/PPP2PPP/RNB1K1NR w KQ - 0 6").reshape(1,769,1)
print(model.predict(_769)[0][0])
_769 = fen_to_769("rnb5/pp1nkr1p/2p5/4pp2/4P3/8/PPP1N1PP/RN3RK1 w - - 2 13").reshape(1,769,1)
print(model.predict(_769)[0][0]) # -4.9
_769 = fen_to_769("rnb5/pp1nkr1p/2p5/4pp2/4P3/8/PPP1N1PP/RN3RK1 b - - 2 13").reshape(1,769,1)
print(model.predict(_769)[0][0])

In [ ]:
#model.evaluate(X_test, y_test)
#model.evaluate(X_test, y_test, batch_size=64)

def calculateCpLoss(testBbs, testScores):
    totalLoss = 0.0
    i = 0
    for bbs in testBbs:
        bbs = bbs.reshape(1,769,1)
        pred = model.predict(bbs, verbose=0)[0][0]
        totalLoss += abs(pred - testScores[i])
        i += 1
    return totalLoss / len(testBbs)

fens, scores = getFensAndScores("sf_d9_data/sf_d9_1000.plain")
X_white, X_black, whiteScores, blackScores = [], [], [], []
i = -1
while True:
    i += 1
    if " w " in fens[i] and len(X_white) < 500:
        X_white.append(fen_to_769(fens[i]))
        whiteScores.append(scores[i])
    elif " b " in fens[i] and len(X_black) < 500:
        X_black.append(fen_to_769(fens[i]))
        blackScores.append(scores[i])
    else:
        break

print("White centipawn loss:", calculateCpLoss(X_white, whiteScores))
print("Black centipawn loss:", calculateCpLoss(X_black, blackScores))

In [ ]:
def printWeightsForCpp(model):
    for layer in model.layers:
        print("------ LAYER", layer.name, "------")
        weights = layer.get_weights()
        if len(weights) > 0:
            rows, cols = weights[0].shape
            if cols > 1:
                print(f"double {layer.name}[{rows}][{cols}] = {{", end="")
                for row in weights[0]:
                    print("{", ",".join(str(val) for val in row), "},", end="")
            else:
                print(f"double {layer.name}[{rows}] = {{", end="")
                for row in weights[0]:
                    print(",".join(str(val) for val in row), ",", end="")

            print("};")
            print()
        print()
        
printWeightsForCpp(model)

In [ ]:
def printBiasesForCpp(model):
    for layer in model.layers:
        print("------ LAYER", layer.name, "------")
        biases = layer.get_weights()[1]
        print(f"double biases[{len(biases)}] = {{", end="")
        print(",".join(str(val) for val in biases), ",")
        print()
        
printBiasesForCpp(model)